In [1]:
import sys  
sys.path.insert(0, '../')

# `Encoder`/`Decoder` example

This notebook illustrates the use of the `Encoder` and `Decoder` class for a 'prepare and measure' QKD protocol.  

Let's start with the necessary imports:

In [2]:
from encoding import Encoder, Decoder
from random import choices

##  - encoding: generating key and basis bits
The first step is generating the bits to encode (`key_bits_A`) and the bits (`basis_bits_A`) that determine the basis used to encode `key_bits_A`.

**NOTE**: in order to have predictable decoding results, the bits in `basis_bits_A` are all zero.  
To try with random generated basis bits, just uncomment line `4` in the following snippet.  

**NOTE**: the first bit in list of key bits is the least significative bit.

In [3]:
key_size = 10
key_bits_A = choices(population=(0,1), k=key_size)
basis_bits_A = [0 for i in range(key_size)]
#base_bits_A = choices(population=(0,1), k=key_size)

print(f'A key bits: {key_bits_A} \
      \nA basis bits: {basis_bits_A}')

A key bits: [1, 1, 0, 0, 0, 1, 0, 1, 0, 1]       
A basis bits: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


## - encoding: instantiating an `Encoder` object
The second step is instantiating an Encoder object and initializing it with the key bits, the basis bits, and the protocol to use (BB84, in this example)

In [4]:
encoder = Encoder(key_bits_A, basis_bits_A, 'BB84')

## - encoding: calling the `encode()` method
To encode the key bits, call `Encoder.encode()` method on the `Encoder` object. 

Under the hood, a (protocol dependent) quantum circuit is generated using the qiskit framework; 
the qubits, initially in the |0> state, are transformed using quantum gates according to the key bits and the basis bits.

For instance, in the case of BB84 protocol: 
- if the i-th key bit is `1`, a qubit-flip gate is applied; 
- if the i-th basis bit is `1`, a Hadamard gate is applied.

The result of encoding is a state vector reflecting the action of the quantum circuit.

**NOTE**: the order used by qiskit to store the qubits is big-endian (the most significant is the leftmost qubit), while the standard notation is with the least significant bit/qubit as the first bit.  

to clarify, encoding the bit string '001' in the computational basis, will result in the state vector |100> instead of the state vector |001>.


In [5]:
encoder.encode()
encoded = encoder.state_vector
encoded.draw('latex')

<IPython.core.display.Latex object>

## - decoding: generate the basis bits
To decode the qubits in the state vector resulting from encoding, the bits to determine in which basis measure each qubit (`basis_bits_B`) is generated;

**NOTE**: the bits in `basis_bits_B` are all zero in order to get predictable decoding results.  
To use random bits, just uncomment line `2` in the following snippet.

In [6]:
basis_bits_B = [0 for i in range(key_size)]
#basis_bits_B = choices(population=(0,1), k=key_size)

print(f'B basis bits: {basis_bits_B}')

B basis bits: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


## - decoding: instantiating a `Decoder` object
then, a `Decoder` object is needed; this is initialized with the state vector, the basis bits, and the protocol to use (BB84, in this example)

In [7]:
decoder = Decoder(encoded, basis_bits_B, 'BB84')

## - decoding: calling the `decode()` method  
To decode the qubits, call the `Decoder.decode()` method on the `Decoder` object.  

Under the hood, a (protocol dependent) quantum circuit is generated using the qiskit framework; 
the qubits, initialized with the state vector, are transformed using quantum gates according to the basis bits.

For instance, in the case of BB84 protocol, if the i-th basis bit is `1`, a Hadamard gate is applied.

The result of decoding is a list of bit, with the least significative bit as first element.

In [8]:
decoder.decode()

key_bits_B = decoder.key_bits
print(f'B key bits: {key_bits_B}')

B key bits: [1, 1, 0, 0, 0, 1, 0, 1, 0, 1]


## - checking the results
the following line check if the decoded bits are the same of the original ones;  
clearly, this makes sense only if the computational basis is used during both encoding and decoding

In [9]:
assert key_bits_A == key_bits_B